# Quartz-Feldspar Geobarometer Prototype &
# Liquidus Phase Diagram Generator Prototype

Required Python packages

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys
import matplotlib.pyplot as plt
%matplotlib inline

Required ThermoEngine packages

In [ ]:
from thermoengine import core, phases, model, equilibrate
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Use rhyolite-MELTS 1.0.2 liquid as the omnicomponent phase

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database(liq_mod='v1.0')
Liquid = modelDB.get_phase('Liq')

### Specify a collection of compatible solid phases

In [ ]:
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')

Set up conversion matrices for translating solid phase (endmember) stoichiometry to liquid components

In [ ]:
elmLiqMat = Liquid.props['element_comp']
idx = np.argwhere(np.all(elmLiqMat[..., :] == 0, axis=0))
conLiqMat = np.linalg.inv(np.delete(elmLiqMat, idx, axis=1))
conLiqMat[np.abs(conLiqMat) < np.finfo(np.float).eps] = 0
conPhs_d = {}
for phs in [Quartz, Feldspar, Spinel, Opx, RhomOx]:
    name = phs.props['phase_name']
    elmPhsMat = phs.props['element_comp']
    elmPhsMat = (np.delete(elmPhsMat, idx, axis=1))
    conPhsMat = np.matmul(elmPhsMat, conLiqMat)
    conPhsMat[np.abs(conPhsMat) < 100*np.finfo(np.float).eps] = 0
    conPhs_d[name] = conPhsMat

### Specify a compatible fluid phase

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Specify a bulk composition for testing
This composition is late erupted Bishop Tuff

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    10.0
}
tot_grm_oxides = 0.0
for key in grm_oxides.keys():
    tot_grm_oxides += grm_oxides[key]

Convert to moles and check feasibility of composition

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

Convert to moles of elements for input to the Equilibrate class

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

## Equilibrate class instance for Geobarometer
- Only liquid and water phases in system
- Oxygen fugacity is constrained using Kress and Carmichael

In [ ]:
phs_sys = [Liquid, Water]
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

This function:
- Is used by a minimizer (below) to refine t and p
- Accepts a bulk composition (in moles of elements) and fO2 offset from NNO
- Equilibrates a liquid with water at the specified fO2
- Calculates a chemical affinity of feldspar saturation and one for quartz saturation
- result all results of affinity calculations (return_for_scipy=False), or
- a sum of squares of the calculated affinities for teh two phases

If feldspar and quartz are liquidus phases at some input t, p, then their affinities are zero. 

In [ ]:
def affinity(x, blk_cmp, NNO_offset, doprint=False, return_for_scipy=True):
    global state, equil, Feldspar, Quartz
    t = x[0] 
    p = x[1]
    if state is None:
        state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=False)
    else:
        state = equil.execute(t, p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=False)
    if doprint:
        state.print_state()
    muLiq = state.dGdn(t=t, p=p, element_basis=False)[:,0]
    result = {}
    muFld = np.array([
        5.0*muLiq[0]/2.0 + muLiq[2]/2.0 + muLiq[11]/2.0,
            muLiq[0] + muLiq[2] + muLiq[10],
        2.0*muLiq[0] + muLiq[12]
    ])
    muQtz = np.array([muLiq[0]])
    result['Feldspar'] = Feldspar.affinity_and_comp(t, p, muFld, method='special')
    result['Quartz']   = Quartz.affinity_and_comp(t, p, muQtz, method='special')
    if return_for_scipy:
        AffnFld = result['Feldspar'][0]
        AffnQtz = result['Quartz'][0]
        sumsq = (AffnFld/13.0)**2 + (AffnQtz/3.0)**2
        print ('x', end='')
        return sumsq
    else:
        return result

Call the function to compute results for some arbitrarily chosen t, p

In [ ]:
t = 1034.0
p = 1750.0
NNO_offset = 0.0
state = None
res_d = affinity(np.array([t, p]), blk_cmp, NNO_offset, doprint=True, return_for_scipy=False)
print ()
for key,value in res_d.items():
    print ('{0:<12.12s} Affn: {1:10.2f} Comp: '.format(key, value[0]), end='')
    print (value[1])

This next call should execute faster because the state *global variable* is used as an initial guess 

In [ ]:
res_d = affinity(np.array([t, p]), blk_cmp, NNO_offset, doprint=True, return_for_scipy=False)
print ()
for key,value in res_d.items():
    print ('{0:<12.12s} Affn: {1:10.2f} Comp: '.format(key, value[0]), end='')
    print (value[1])

## Perform the Geobarometer calculation
Use a minimizer from SciPy optimize to find values of t,p that minimize the sum-of-squares of the affinity
- The function result of this minimization should be "zero" if both phases are simultaneously on the liquidus

In [ ]:
t = 1034.0
p = 1750.0
NNO_offset = 0.0
state = None
result = opt.minimize(affinity, 
             np.array([t, p]), 
             args=(blk_cmp, NNO_offset),
             options={'disp':True, 'xatol':1.0, 'fatol':1.0, 'return_all':True},
             method='Nelder-Mead'
            )
result

### Check the result by calling the affinity function directly

In [ ]:
print ('T =', result.x[0]-273.15, '°C, P =', result.x[1], 'bars')
res_d = affinity(result.x, blk_cmp, NNO_offset, doprint=True, return_for_scipy=False)
print ()
for key,value in res_d.items():
    print ('{0:<12.12s} Affn: {1:10.2f} Comp: '.format(key, value[0]), end='')
    print (value[1])

### Check the result by plotting convergence iterates

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.title('Temperature Convergence')
for i,row in enumerate(np.array(result.allvecs)):
    plt.plot(i, row[0]-273.15, 'ro')
plt.xlabel('Iteration')
plt.ylabel('T °C')
plt.subplot(1,2,2)
plt.title('Pressure Convergence')
for i,row in enumerate(np.array(result.allvecs)):
    plt.plot(i, row[1], 'ro')
plt.xlabel('Iteration')
plt.ylabel('P bars')
plt.tight_layout()
plt.show()

## Are quartz and feldspar the true liquidus phases?
- Create a rhyolite-MELTS instance with all potential phases instantiated
- Use the calculated t,p is compute a stable phase assemblage.
- Is the stable assemblage quartz, feldspar and water? Yes, then we are done. No, then the baraometer does not apply

In [ ]:
equilTest = equilibrate.Equilibrate(elm_sys, [Liquid, Feldspar, Quartz, Spinel, Opx, RhomOx, Water])
state = equilTest.execute(result.x[0], result.x[1], bulk_comp=blk_cmp, con_deltaNNO=0.0, debug=0, stats=False)
state.print_state()

## Is there a second feldspar on the liquidus?
- Run the "phase stability" method on the feldspar just computed to see if that composition unmixes
- If compositions are returned that are different from the input composition, and the function value associated with that composition is near zero or negative, then unmixing is likely

In [ ]:
comp_feldspar = state.compositions(phase_name='Feldspar', units='mole_frac')
result_l = Feldspar.determine_phase_stability(result.x[0], result.x[1], comp_feldspar, return_all_results=True)
print ('{0:>10.10s} {1:>7.7s}  '.format('func', 'norm'), end='')
for xend in Feldspar.props['endmember_name']:
    print ('{0:>7.7s} '.format(xend), end='')
print ('')
print ('{0:20.20s}'.format(''), end='')
for xend in comp_feldspar:
    print ('{0:7.4f} '.format(xend), end='')
print (' Reference composition')
for res in result_l:
    print ('{0:10.3g} {1:7.4f}  '.format(res[0], np.linalg.norm(res[1]-comp_feldspar)), end='')
    for xend in res[1]:
        print ('{0:7.4f} '.format(xend), end='')
    print ('')

## Verify a second feldspar
- Lower t by s "small amount" and recompute equilibrium. 
- If a feldspar show sup, the what we have found is a quartz + two feldspar pressure

In [ ]:
state = equilTest.execute(result.x[0]-.5, result.x[1], bulk_comp=blk_cmp, con_deltaNNO=0.0, debug=0, stats=False)
state.print_state()

# Liquidus Phase Diagrams

Liquidus saturation curve method for scalar root finder:  
- Specify t, p, bulk composition, fO2 offset, and a phase instance
- Returns the chemical affinity (and composition) of that phase

In [ ]:
def saturation_curve(t, p, blk_cmp, NNO_offset, phase_obj, doprint=False, return_for_scipy=True):
    global state, equil, conPhs_d
    if state is None:
        state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=False)
    else:
        state = equil.execute(t, p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=False)
    if doprint:
        state.print_state()
    muLiq = state.dGdn(t=t, p=p, element_basis=False)[:,0]
    mLiq  = state.compositions(phase_name='Liquid')
    phase_name = phase_obj.props['phase_name']
    assert phase_name in conPhs_d, phase_name+" is not in conPhs_d"
    muPhase = []
    conPhsMat = conPhs_d[phase_name]
    for i in range(0,conPhsMat.shape[0]):
        sum = 0.0
        for j in range (0,conPhsMat.shape[1]):
            if conPhsMat[i,j] != 0 and mLiq[j] == 0:
                sum = 0.0
                break
            sum += conPhsMat[i,j]*muLiq[j]
        muPhase.append(sum)
    muPhase = np.array(muPhase)
    result = phase_obj.affinity_and_comp(t, p, muPhase, method='special')
    if return_for_scipy:
        return result[0] 
    else:
        return result

## Minimizer to zero chemical affinity by changing T
- in a loop over phases and an inner loop over P
  - find the T that zeroes the affinitry for the phase 
- collect results and plot the diagram
- accuarcy is ± 0.1 °C

In [ ]:
t = 1050
NNO_offset = 0.0
fig = plt.figure(figsize=(15,10))
colors = ['r-', 'g-', 'b-', 'y-', 'm-']
for ic,phs in enumerate([Quartz, Feldspar, Spinel, Opx, RhomOx]): 
    x = []
    y = []
    tk = t
    state = None
    print (phs.props['phase_name'])
    for ip in range(0,21):
        y.append(3000-ip*100)
        result = opt.root_scalar(saturation_curve, 
                                 bracket=(500,2000), 
                                 x0=tk, x1=tk-25, 
                                 xtol=.1, 
                                 args=(3000-ip*100, blk_cmp, NNO_offset, phs), 
                                 method='secant') # secant is fastest
        tk = result.root
        x.append(tk-273.15)
        print ('{0:6.1f}'.format(tk-273.15), end=' ')
    print ()
    plt.plot(np.array(x), np.array(y), colors[ic], label=phs.props['phase_name'])
plt.title('Liquidus Phase Diagram')
plt.xlabel('T °C')
plt.ylabel('P bars')
plt.legend()
plt.show()